### Saving and Loading models - ``TensorFlow``


In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets
import numpy as np

### Configuring the ``device`` for the environment.


In [2]:
physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)

### Let's create a model that will train on the `MNIST` dataset.

In [3]:
(X_train, y_train), (X_test, y_test) = datasets.mnist.load_data()
X_train.shape

(60000, 28, 28)

In [4]:
def normalize(image):
  image = tf.convert_to_tensor(image.astype('float32'))/255
  return image

In [5]:
X_train_tensors =tf.convert_to_tensor(list(map(normalize, X_train)))
X_test_tensors = tf.convert_to_tensor(list(map(normalize, X_test)))

y_test_tensors = tf.convert_to_tensor(y_test)
y_train_tensors = tf.convert_to_tensor(y_train)

In [6]:
y_test_tensors[:2]

<tf.Tensor: shape=(2,), dtype=uint8, numpy=array([7, 2], dtype=uint8)>

In [7]:
model = keras.Sequential([
      keras.layers.Input(shape=(28, 28,)),
      keras.layers.Flatten(),
      keras.layers.Dense(64, activation="relu"),
      keras.layers.Dense(128, activation="relu"),
      keras.layers.Dense(10, activation="softmax")
])

model.compile(loss=keras.losses.SparseCategoricalCrossentropy(),
              optimizer="adam",
              metrics=["accuracy"]
              )
model.fit(X_train_tensors, y_train_tensors, epochs=5, verbose=1, batch_size=32, validation_data=(X_test_tensors, y_test_tensors))
model.summary()

Epoch 1/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.2747 - accuracy: 0.9196 - val_loss: 0.1494 - val_accuracy: 0.9557
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1227 - accuracy: 0.9619 - val_loss: 0.1134 - val_accuracy: 0.9637
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0867 - accuracy: 0.9730 - val_loss: 0.0954 - val_accuracy: 0.9693
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0671 - accuracy: 0.9783 - val_loss: 0.0895 - val_accuracy: 0.9733
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0545 - accuracy: 0.9829 - val_loss: 0.0887 - val_accuracy: 0.9734
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
d

#### Saving model weights
To save the model weights we call `model.save_weights(path)` method. So if we want to load the weights to the model we just call `model.save_weights`.

#### Saving `weights`

```python
model.save_weights('checkpoints/')
```

#### Loading the weights to the model.

```python
model.load_weights('checkpoints/')
```

In [8]:
model.save_weights("checkpoints/")
print("Weights saved")

Weights saved


> When we call the `model.load_weights` then the model will continue trainning from the loaded previous weights.

In [9]:
model.load_weights('checkpoints/')
print("Mode Weights loaded")

Mode Weights loaded


In [10]:
model.compile(loss=keras.losses.SparseCategoricalCrossentropy(),
              optimizer="adam",
              metrics=["accuracy"]
              )
model.fit(X_train_tensors, y_train_tensors, epochs=5, verbose=1, batch_size=32, validation_data=(X_test_tensors, y_test_tensors))

Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0461 - accuracy: 0.9848 - val_loss: 0.0897 - val_accuracy: 0.9745
Epoch 2/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0394 - accuracy: 0.9872 - val_loss: 0.0904 - val_accuracy: 0.9735
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0316 - accuracy: 0.9894 - val_loss: 0.0882 - val_accuracy: 0.9739
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0274 - accuracy: 0.9905 - val_loss: 0.1012 - val_accuracy: 0.9736
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0244 - accuracy: 0.9917 - val_loss: 0.0928 - val_accuracy: 0.9754


> We can esentially see that the model's first epoch accuracy is simmilar or close to the previous model. Meaning this model just continued trainning from where it left of it's 5th epoch.

#### Model Serialization.
This means saving the whole model. We can do that by calling `model.save(path)`. 

#### Serializing Model
```python
model.save("my_model.h5")
```
#### Loading the Serialized model.


```python
new_model = keras.models.load_model("my_model.h5")
```

In [11]:
model.save("my_model.h5")
print("Model Saved.")

Model Saved.


### Evaluating our loaded model.

In [12]:
new_model = keras.models.load_model("my_model.h5")
new_model.evaluate(X_test_tensors, y_test_tensors, batch_size=32, verbose=1)

313/313 [==============================] - 1s 2ms/step - loss: 0.0928 - accuracy: 0.9754


[0.09283297508955002, 0.9753999710083008]

> So we are getting the same accuracy from the previous model.

#### Saving models to json.
We can also save the model weights as json. Let's take a look at the following example:

* [Ref](https://machinelearningmastery.com/save-load-keras-deep-learning-models/#:~:text=Keras%20separates%20the%20concerns%20of,different%20formats%3A%20JSON%20and%20YAML.)

#### Serialize model to ``JSON``

In [14]:
model_json = model.to_json() # convert the model to json.

with open('model_json.json', 'w') as f:
  f.write(model_json)
print("Model Saved.")

Model Saved.


#### Loading the `JSON` model.

In [16]:
json_file = open('model_json.json', 'r')
model_json_loaded = json_file.read()
json_file.close()
loaded_model = keras.models.model_from_json(model_json_loaded)
print(loaded_model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                50240     
_________________________________________________________________
dense_1 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
Total params: 59,850
Trainable params: 59,850
Non-trainable params: 0
_________________________________________________________________
None


### Serialize model to ``YAML``
This is simmilar to saving and loading the model in json.

In [17]:
model_yaml = model.to_yaml() # convert the model to yaml.

with open('model_yaml.yaml', 'w') as f:
  f.write(model_yaml)
print("Model Saved.")

Model Saved.


In [19]:
yaml_file = open('model_yaml.yaml', 'r')
model_yaml_loaded = yaml_file.read()
yaml_file.close()
loaded_model = keras.models.model_from_yaml(model_yaml_loaded)
print(loaded_model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                50240     
_________________________________________________________________
dense_1 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
Total params: 59,850
Trainable params: 59,850
Non-trainable params: 0
_________________________________________________________________
None


> That's more of it about loading and saving model and model weights.